In [1]:
import torch, sys, os, random, time
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split, TensorDataset, Subset
import numpy as np
from datetime import datetime

# Append the path
src_path = os.path.abspath('../src')
if src_path not in sys.path:
    sys.path.append(src_path)
    
import fl
from models import *
from quantization import *
from client import Client, ClientResources

# Hyperparameters

In [2]:
# Hyperparameters
hp = {
    "run_id": "results",
    "run_number": "noniid2_standard_noshared",
    "num_classes": 10,  # 10 for MNIST
    "learning_rate": 1e-3,
    "batch_size": 32,
    "num_clients": 100,
    "num_rounds": 50,
    "epochs": 5,    # number of epochs to train in each round
    "setup": "standard",    # ["standard", "scalar", "kure", "mqat"]
    "split": "NONIID",   # ["RANDOM", "NONIID"]
    "classes_per_client": 2,
    "use_client_selection": True,
    "quantize": False,
    "lambda_kure": 1e-4,
    "delta": 1e-2,
    "shared_fraction": 0.00,
    "deadline": 1000.0,
    "bit_widths": [8, 16, 32]
}

# Load the Tensorboard notebook

In [3]:

import tensorflow as tf
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter(log_dir=".")
"""%load_ext tensorboard
%tensorboard --logdir=."""

'%load_ext tensorboard\n%tensorboard --logdir=.'

In [4]:
shared_fraction = hp['shared_fraction']
num_clients = hp["num_clients"]
batch_size = hp["batch_size"]
classes_per_client = hp["classes_per_client"]
num_classes = hp["num_classes"]
lr = hp["learning_rate"]
epochs = hp["epochs"]
split = hp["split"]
lambda_kure = hp["lambda_kure"]
delta = hp["delta"]
setup = hp["setup"]
use_client_selection = hp["use_client_selection"]
quantize = hp["quantize"]
bit_widths = hp["bit_widths"]

## Obtain Dataset

In [5]:
from torchvision import datasets, transforms

# Define transformations for the dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Normalize to range [-1, 1]
])

# Download and load MNIST dataset
mnist_full = datasets.MNIST(root="../data", train=True, transform=transform, download=True)
mnist_test = datasets.MNIST(root="../data", train=False, transform=transform, download=True)

# Split training data into training and validation sets
train_size = int(0.8 * len(mnist_full))  # 80% for training
val_size = len(mnist_full) - train_size  # 20% for validation
train_dataset, val_dataset = random_split(mnist_full, [train_size, val_size])

shared_size = int(len(train_dataset) * shared_fraction)
remaining_size = len(train_dataset) - shared_size
train_shared_dataset, train_dataset = random_split(train_dataset, [shared_size, remaining_size])

# Extract indices from Subset objects
train_indices = train_dataset.indices  # List of training indices

train_shared_indices = train_shared_dataset.indices if shared_fraction > 0.0 else []  # List of training indices
val_indices = val_dataset.indices      # List of validation indices

# Create training and validation MNIST datasets
mnist_train = datasets.MNIST(root="../data", train=True, transform=transform, download=False)
mnist_train_shared = datasets.MNIST(root="../data", train=True, transform=transform, download=False)
mnist_val = datasets.MNIST(root="../data", train=True, transform=transform, download=False)

# Filter datasets by indices
mnist_train.data = mnist_train.data[torch.tensor(train_indices)]
mnist_train.targets = mnist_train.targets[torch.tensor(train_indices)]

if shared_fraction > 0.0:
    mnist_train_shared.data = mnist_train_shared.data[torch.tensor(train_shared_indices)]
    mnist_train_shared.targets = mnist_train_shared.targets[torch.tensor(train_shared_indices)]
else:
    mnist_train_shared = []

mnist_val.data = mnist_val.data[torch.tensor(val_indices)]
mnist_val.targets = mnist_val.targets[torch.tensor(val_indices)]

# Print dataset sizes
print(f"Train dataset size: {len(mnist_train)}, Train Shared dataset size: {len(mnist_train_shared)}, Validation dataset size: {len(mnist_val)}, Test dataset size: {len(mnist_test)}")

Train dataset size: 48000, Train Shared dataset size: 0, Validation dataset size: 12000, Test dataset size: 10000


# Split Dataset: non-IID / Random

### Create client train and validation datasets

In [6]:
from collections import Counter

# Number of clients and non-IID split
train_data_size = len(mnist_train) // num_clients
val_data_size = len(mnist_val) // num_clients

if split == "NONIID":
    
    # Create indices for each class
    train_class_indices = {i: np.where(np.array(mnist_train.targets) == i)[0] for i in range(num_classes)}
    val_class_indices = {i: np.where(np.array(mnist_val.targets) == i)[0] for i in range(num_classes)}

    train_indices = []  # Initialize training indices
    val_indices = []    # Initialize validation indices

    # Randomize the order of classes for validation
    shuffled_classes = np.random.permutation(num_classes)

    for client_id in range(num_clients):
        # Choose random classes for training
        chosen_train_classes = np.random.choice(num_classes, classes_per_client, replace=False)
        train_client_idx = []

        # Assign validation classes independently
        chosen_val_classes = np.random.choice(shuffled_classes, classes_per_client, replace=False)
        val_client_idx = []

        for cls in chosen_train_classes:
            train_cls_size = len(train_class_indices[cls]) // (num_clients // classes_per_client)
            train_cls_idx = np.random.choice(train_class_indices[cls], train_cls_size, replace=False)
            train_client_idx.extend(train_cls_idx)
            train_class_indices[cls] = np.setdiff1d(train_class_indices[cls], train_cls_idx)  # Avoid duplication

        for cls in chosen_val_classes:
            val_cls_size = len(val_class_indices[cls]) // (num_clients // classes_per_client)
            val_cls_idx = np.random.choice(val_class_indices[cls], val_cls_size, replace=False)
            val_client_idx.extend(val_cls_idx)
            val_class_indices[cls] = np.setdiff1d(val_class_indices[cls], val_cls_idx)  # Avoid duplication

        train_indices.append(train_client_idx)
        val_indices.append(val_client_idx)

    # Create datasets and DataLoaders for each client
    train_datasets = [Subset(mnist_train, indices) for indices in train_indices]
    train_loaders = [DataLoader(ds, batch_size=batch_size, shuffle=True) for ds in train_datasets]

    val_datasets = [Subset(mnist_val, indices) for indices in val_indices]
    val_loaders = [DataLoader(ds, batch_size=batch_size, shuffle=True) for ds in val_datasets]

else:
    
    # Split the training data into smaller datasets for each client
    train_datasets = random_split(mnist_train, [train_data_size] * num_clients)
    train_loaders = [DataLoader(ds, batch_size=batch_size, shuffle=True) for ds in train_datasets]
    val_datasets = random_split(mnist_train, [train_data_size] * num_clients)
    val_loaders = [DataLoader(ds, batch_size=batch_size, shuffle=True) for ds in train_datasets]

# val_loaders = DataLoader(mnist_val, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(mnist_test, batch_size=batch_size, shuffle=False)


print(f"Simulated {num_clients} clients, each with {train_data_size} training samples, and {len(mnist_val)} validation samples")
# Debugging: Output distribution of classes for all clients
print("Training Set Label Distribution:")
for i, client_dataset in enumerate(train_datasets):
    client_loader = torch.utils.data.DataLoader(client_dataset, batch_size=len(client_dataset))
    client_samples, client_labels = next(iter(client_loader))
    label_counts = Counter(client_labels.tolist())
    print(f"Client {i} label distribution: {dict(label_counts)}")

print("\nValidation Set Label Distribution:")
for i, client_dataset in enumerate(val_datasets):
    client_loader = torch.utils.data.DataLoader(client_dataset, batch_size=len(client_dataset))
    client_samples, client_labels = next(iter(client_loader))
    label_counts = Counter(client_labels.tolist())
    print(f"Client {i} label distribution: {dict(label_counts)}")

Simulated 100 clients, each with 480 training samples, and 12000 validation samples
Training Set Label Distribution:
Client 0 label distribution: {2: 95, 0: 94}
Client 1 label distribution: {5: 86, 2: 93}
Client 2 label distribution: {2: 91, 4: 93}
Client 3 label distribution: {1: 109, 5: 84}
Client 4 label distribution: {2: 89, 7: 99}
Client 5 label distribution: {4: 91, 0: 92}
Client 6 label distribution: {6: 94, 0: 90}
Client 7 label distribution: {2: 87, 9: 96}
Client 8 label distribution: {4: 89, 1: 107}
Client 9 label distribution: {9: 94, 1: 105}
Client 10 label distribution: {0: 89, 3: 97}
Client 11 label distribution: {8: 93, 1: 102}
Client 12 label distribution: {3: 95, 0: 87}
Client 13 label distribution: {8: 91, 4: 87}
Client 14 label distribution: {1: 100, 5: 83}
Client 15 label distribution: {1: 98, 2: 86}
Client 16 label distribution: {0: 85, 5: 81}
Client 17 label distribution: {7: 97, 5: 79}
Client 18 label distribution: {0: 83, 8: 89}
Client 19 label distribution: {2:

# Optionally distribute the shared data among clients

In [7]:
if shared_fraction > 0.0:
    alpha = 0.05    # Each client gets partition of the shared data

    shared_loader = torch.utils.data.DataLoader(mnist_train_shared, batch_size=len(mnist_train_shared))
    shared_samples, shared_labels = next(iter(shared_loader))
    num_shared_samples = int(alpha * len(shared_samples))

    merged_train_datasets = []
    for client_dataset in train_datasets:
        client_loader = torch.utils.data.DataLoader(client_dataset, batch_size=len(client_dataset))
        client_samples, client_labels = next(iter(client_loader))
        
        # Combine shared and local data
        combined_samples = torch.cat([client_samples, shared_samples[:num_shared_samples]])
        combined_labels = torch.cat([client_labels, shared_labels[:num_shared_samples]])
        merged_train_dataset = torch.utils.data.TensorDataset(combined_samples, combined_labels)
        merged_train_datasets.append(merged_train_dataset)

    train_loaders = [DataLoader(ds, batch_size=batch_size, shuffle=True) for ds in merged_train_datasets]

    # Display the label distribution for each client
    for i, client_dataset in enumerate(merged_train_datasets):
        client_loader = torch.utils.data.DataLoader(client_dataset, batch_size=len(client_dataset))
        client_samples, client_labels = next(iter(client_loader))
        label_counts = Counter(client_labels.tolist())
        print(f"Client {i} label distribution: {dict(label_counts)}")

## Set up and initialize the Global Model

In [8]:
# 1. Initialization: Instantiate the global model (server)
global_model = QuantStubModel(q=quantize)
if(quantize):
    global_model.qconfig = torch.quantization.get_default_qat_qconfig("fbgemm")
    torch.quantization.prepare_qat(global_model, inplace=True)

# Clients

### Client Creation
We create clients with various resources, in which we take into account, speed, availability, battery level, bandwidth, etc.

In [9]:
clients = []
for i in range(num_clients):
    mock_resources = ClientResources.generate_random((len(train_datasets[i]), len(train_datasets[i])))
    new_client = Client(id=i, resources=mock_resources, dataset=train_datasets[i], dataloader=train_loaders[i], val_loader=val_loaders[i])
    clients.append(new_client)


In [10]:
def client_selection_with_constraints(clients, deadline):

    # Calculate update times for all clients
    client_times = [
        (client, client.resources.dataset_size / client.resources.speed_factor)  # (Client object, update_time)
        for client in clients
    ]
    
    # Sort clients by their update time
    client_times.sort(key=lambda x: x[1])
    selected_clients = []
    #total_time = 0

    #print(deadline)
    # Select clients within the deadline
    for client, update_time in client_times:
        #print('ut:', update_time)
        if update_time <= deadline:
            selected_clients.append(client)
            #total_time += update_time
        else:
            break  # Stop once adding a client exceeds the deadline

    # TODO: fix client choice
    if len(selected_clients) == 0:
        selected_clients.append(client)

    return selected_clients

def select_indices(n, k):
    return random.sample(range(n), k)

# Validation

In [11]:
def validate_model(global_model, val_loaders):

    global_model.eval()  # Set the model to evaluation mode
    total_loss = 0.0
    total_correct = 0
    total_samples = 0
    criterion = torch.nn.CrossEntropyLoss()

    with torch.no_grad():
        for val_loader in val_loaders:
            client_loss = 0.0
            client_correct = 0
            client_samples = 0

            for inputs, labels in val_loader:
                outputs = global_model(inputs)
                loss = criterion(outputs, labels)
                client_loss += loss.item() * len(labels)  # Accumulate loss weighted by batch size
                _, preds = torch.max(outputs, 1)
                client_correct += (preds == labels).sum().item()  # Count correct predictions
                client_samples += len(labels)  # Track number of samples

            # Accumulate client metrics into global metrics
            total_loss += client_loss
            total_correct += client_correct
            total_samples += client_samples

    # Compute global averages
    avg_loss = total_loss / total_samples if total_samples > 0 else 0.0
    avg_accuracy = total_correct / total_samples if total_samples > 0 else 0.0
    return avg_loss, avg_accuracy


## Run Federated Learning Round

In [12]:
# Wrapper function for client training
def train_client_parallel(client, global_model, epochs=1, lr=0.001, quantize=False, lambda_kure=0.0, delta=0.0, setup='standard',bit_widths=[32]):
    #print(f"Training client {client.id} with resources {client.resources}")
    return client.id, client.train(global_model, epochs, lr, quantize, lambda_kure, delta, setup, bit_widths)

In [13]:
from concurrent.futures import ThreadPoolExecutor, as_completed

# Generate a unique log directory based on the current time
run_number = hp["run_number"] + datetime.now().strftime('-%m-%d-%H-%M')
log_dir = f"./logs/{hp['run_id']}/run_{run_number}-{datetime.now().strftime('%m-%d-%H-%M')}"
writer = SummaryWriter(log_dir=log_dir)
hyperparams_text = "\n".join([f"{key}: {value}" for key, value in hp.items()])
start_time = time.time()

# Federated Learning with FedCS Client Selection
num_rounds = hp["num_rounds"]
epochs = hp["epochs"]
round_deadline = hp["deadline"]  # Example round deadline (in ARBITRARY time units)
print("GPU available:", torch.cuda.is_available())

GPU available: True


In [ ]:
# Conduct federated learning rounds
for round_num in range(num_rounds):
    print(f"Round {round_num + 1}")
    
    # 2. Resource Request
    k = random.randint(1, num_clients)
    resource_requested_clients = random.sample(range(num_clients), k)
    requested_clients = [clients[i] for i in resource_requested_clients]
    
    # 3. Client Selection: Collect client updates
    if use_client_selection:
        selected_train_clients = client_selection_with_constraints(requested_clients, round_deadline)
    else:
        selected_train_clients = requested_clients
    print(f"-> Resource requested from {len(resource_requested_clients)} clients, {len(selected_train_clients)} clients fulfilled the criteria")

    client_states = []
    total_loss = 0
    num_batches = len(selected_train_clients)
    # Parallelize client training
    with ThreadPoolExecutor(max_workers=len(selected_train_clients)) as executor:
        futures = {
            executor.submit(
                train_client_parallel, 
                client, 
                global_model, 
                epochs=1,
                lr=lr,
                quantize=quantize,
                lambda_kure=lambda_kure,
                delta=delta,
                setup=setup,
                bit_widths=bit_widths
                ): client
            for client in selected_train_clients
        }
        
        for future in as_completed(futures):
            client_id, (client_state, client_loss) = future.result()
            # print(f"Client {client_id} completed training.")
            client_states.append(client_state)
            total_loss += client_loss

    # Compute average training loss
    avg_round_training_loss = total_loss / num_batches if num_batches > 0 else 0
    writer.add_scalar("Metrics/Training loss", avg_round_training_loss, round_num + 1)
    print(f"Training loss after round {round_num + 1}: {avg_round_training_loss}")

    # 6. Aggregation: Aggregate updates using Federated Averaging
    new_global_state = fl.federated_averaging(global_model, client_states)  
    global_model.load_state_dict(new_global_state)
    print(f"Global model updated for round {round_num + 1}")

    # 7. Evaluate on Validation Set
    val_loaders = [client.val_loader for client in clients]  # Get all validation loaders
    val_loss, val_accuracy = validate_model(global_model, val_loaders)
    print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2%}")
    
    # Optional: Log metrics for visualization
    writer.add_scalar("Metrics/Validation Loss", val_loss, round_num + 1)
    writer.add_scalar("Metrics/Validation Accuracy", val_accuracy, round_num + 1)

if quantize:
    torch.quantization.convert(global_model, inplace=True)


Round 1
-> Resource requested from 8 clients, 8 clients fulfilled the criteria
Training loss after round 1: 1.1212337803095578
Global model updated for round 1
Validation Loss: 2.6049, Validation Accuracy: 15.77%
Round 2
-> Resource requested from 65 clients, 65 clients fulfilled the criteria
Training loss after round 2: 1.119649853307904
Global model updated for round 2
Validation Loss: 2.0955, Validation Accuracy: 27.44%
Round 3
-> Resource requested from 43 clients, 43 clients fulfilled the criteria
Training loss after round 3: 0.8739681875153736
Global model updated for round 3
Validation Loss: 1.9325, Validation Accuracy: 43.52%
Round 4
-> Resource requested from 91 clients, 91 clients fulfilled the criteria


## Evaluate Model

In [ ]:
def evaluate_model(model, dataloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    test_accuracy = correct / total
    print(f"Global model accuracy: {test_accuracy:.2%}")
    return test_accuracy
    
# Evaluate the model on the test dataset
test_accuracy = evaluate_model(global_model, test_loader)

# End TensorBoard writer
final_metrics = {}
hp_cleaned = {k: v for k, v in hp.items() if isinstance(v, (int, float, str, bool, torch.Tensor))}
writer.add_hparams(hp_cleaned, final_metrics)
writer.close()

Global model accuracy: 79.13%


In [ ]:
# Save the final model
model_save_path = f"./saved_models/global_model_{hp['run_id']}_{run_number}.pth"
torch.save(global_model.state_dict(), model_save_path)
print(f"Global model saved at {model_save_path}")

Global model saved at ./saved_models/global_model_results_noniid2_standard_noshared-12-05-03-45.pth
